<h1> Introduction </h1>

<div> Training performance is not just a function of the GPU computational power but the efficiency of the end to end training pipeline. The goal of this lab is to:
    <ul>
        <li> Illustrate the differences in training performance introduced by the limitations of the data loading and augmentation pipeline. </li>
        <li> Illustrate how individual components of the pipeline affect the overall performance.</li>
        <li> Demonstrate how other factors such as networking affect the training performance.</li>
    </ul>
</div>

<div>Before we dive into the exercise lets execute a warmup script that will prepare the computer resources for the exercise.</div>

In [ ]:
# Warmup script
!mpirun -np 4 \
    --allow-run-as-root \
    -H localhost:4 \
    -bind-to none -map-by slot \
    -x NCCL_DEBUG=INFO -x PATH \
    -mca pml ob1 -mca btl ^openib \
    python3 cnn/googlenet.py --precision=fp16 --batch_size=20 --data_dir=/dli/data/tfdata352p90per2

<div>Note that in this notebook we do not execute our code directly, instead, the code is started using the MPI framework. MPI is commonly used in High Performance Computing and a tool of choice to start distributed workloads. Let us have a look at the structure of the below command:
    <ul>
        <li> The <b>mpirun</b> command uses the MPI framework to execute our code in a distributed environment. </li>
        <li> The <b>-np</b> option tells the MPI framework how many processes we want executed in parallel. Since we will further configure our Horovod environment to use one GPU per process this will also indicate the number of GPUs used in training. </li>
        <li> The <b>-H</b> option defines the way we would like our workload distributed across our cluster. Since in this case we have one computer with 4 GPUs we will specify "localhost:4" indicating that we would like all 4 of our processes to run locally.</li>
        <li><b>-bind-to none -map-by slot</b> option prevents MPI from binding our process to a single CPU core which (as discussed in the lecture) could significantly hurt performance.</li>
        <li><b>-x NCCL_DEBUG=INFO</b> option will display NCCL communication debug information.</li>
        <li><b>-mca pml ob1 -mca btl ^openib</b> options force the use of TCP for MPI communication which is not handled by NCCL (NCCL is handling gradient synchronisation but your code might also implement other communication logic through the use of hvd.broadcast() and hvd.allgather()). </li>
        <li><b>python3 cnn/googlenet.py --precision=fp16 --batch_size=20 --data_dir=/dli/data/tfdata352p90per2</b> defines the executable we are asking MPI to launch together with parameters specific to our python program</li>
     </ul>
    For more information about the flags used and overall best practice for running Horovod please refer to <a href="https://github.com/uber/horovod/blob/master/docs/running.md">the following link</a>.
 </div>


<h1> GPU vs the pipeline performance</h1>

<div> During the lecture, we have discussed the challenge of building efficient deep learning solutions that allow for linear scaling (with the number of GPUs). Linear scaling is critical for achieving short training time and effectively utilise resources in our infrastructure. </div>
<br/>
<div>The goal of the first part of this lab is to illustrate the impact of the data loading and augmentation pipeline on the real training performance. In particular, we will execute a number of training jobs using synthetic data (limiting the training process to the GPU) and compare the performance to the training job involving the state of the art end to end data pipeline.</div>
<br/>

<div> Let's start with a googlenet network trained on the synthetic data: </div>

In [ ]:
# We are running googlenet benchmarks in isolation from the data loading and augmentation process
!mpirun -np 4 \
    --allow-run-as-root \
    -H localhost:4 \
    -bind-to none -map-by slot \
    -mca pml ob1 -mca btl ^openib \
    python3 cnn/googlenet.py --precision=fp16 --batch_size=896 --num_iter=50

In [ ]:
# Please enter the performance extracted from the above experiment (images per second)
googlenet_gpu = 6446.6 #Type your value as a floating point number

In [ ]:
# We execute the same code but include the real data as a consequence
!mpirun -np 4 \
    --allow-run-as-root \
    -H localhost:4 \
    -bind-to none -map-by slot \
    -mca pml ob1 -mca btl ^openib \
    python3 cnn/googlenet.py --precision=fp16 --batch_size=896 --num_iter=50 --data_dir=/dli/data/tfdata352p90per2

In [ ]:
googlenet_pipeline = 3453.7#Type your value as a floating point number

<div> We have observed a degradation in the end to end training performance. Let's repeat this exercise for a couple other neural networks and plot them to see how the effect is affected by neural network model properties. </div>

<br/>

<div>Throughout the next segment of the Lab, we will run the same benchmark for the AlexNet, ResNet50 and ResNet152 networks. Feel free to execute the code to gather the actual figures on your system. Alternatively, the figures were collected already on the same machine type so feel free to skip the execution of the training code up to the next section marked with "End of the optional segment" if you are not interested in the dynamics of the execution process. If you decide to skip the optional section make sure you execute the code section below. This code will assign the performance values to the variables used for plotting the results on a graph similar to the one covered in the lecture (illustrating rooftop analysis).</div>

In [ ]:
# If you decide not to execute the optional sections please execute this cell
# to assign performance values used later for performance visualisation
resnet50_gpu = 2795.8
resnet50_pipeline = 2741.2
resnet152_gpu = 1250.5
resnet152_pipeline = 1183.9
alexnet_gpu = 26790.8
alexnet_pipeline = 3250.0

<h1>Optional segment</h1>

<br/>
<div> Let's continue the data collection, this time by executing ResNet50:</div>

In [ ]:
# We are running ResNet50 benchmarks in isolation from the data loading and augmentation process
!mpirun -np 4 --allow-run-as-root -H localhost:4 -bind-to none -map-by slot -x NCCL_DEBUG=INFO -x PATH -mca pml ob1 -mca btl ^openib python3 cnn/resnet.py --layers=50 --precision=fp16 --batch_size=272

In [ ]:
resnet50_gpu = 2795.8 #Type your value as a floating point number 

In [ ]:
# And now end to end ResNet50 training
!mpirun -np 4 --allow-run-as-root -H localhost:4 -bind-to none -map-by slot -x NCCL_DEBUG=INFO -x PATH -mca pml ob1 -mca btl ^openib python3 cnn/resnet.py --layers=50 --precision=fp16 --batch_size=272 --data_dir=/dli/data/tfdata352p90per2

In [ ]:
resnet50_pipeline = 2741.2#Type your value as a floating point number

and ResNet152:

In [ ]:
# ResNet152 GPU only
!mpirun -np 4 --allow-run-as-root -H localhost:4 -bind-to none -map-by slot -x NCCL_DEBUG=INFO -x PATH -mca pml ob1 -mca btl ^openib python3 cnn/resnet.py --layers=152 --precision=fp16 --batch_size=128

In [ ]:
resnet152_gpu = 1250.5 #Type your value as a floating point number

In [ ]:
# And the full pipeline
!mpirun -np 4 --allow-run-as-root -H localhost:4 -bind-to none -map-by slot -x NCCL_DEBUG=INFO -x PATH -mca pml ob1 -mca btl ^openib python3 cnn/resnet.py --layers=152 --precision=fp16 --batch_size=128 --data_dir=/dli/data/tfdata352p90per2

In [ ]:
resnet152_pipeline = 1183.9 #Type your value as a floating point number

<div> We can clearly see that there is a relationship between the neural network computational complexity and the impact the end to end pipeline has on efficiency. This relationship is very intuitive. The more time the GPU needs to spend processing a single image (the fewer images we are processing per second) the less we need to deliver through our pipeline. </div>
    
<br/>

<div> To illustrate this phenomenon let's observe the behaviour of one of the simplest neural networks - AlexNet: </div>

In [ ]:
# We are running alexnet benchmarks in isolation from the data loading and augmentation process
!mpirun -np 4 --allow-run-as-root -H localhost:4 -bind-to none -map-by slot -x NCCL_DEBUG=INFO -x PATH -mca pml ob1 -mca btl ^openib python3 cnn/alexnet.py --precision=fp16 --batch_size=512

In [ ]:
alexnet_gpu = 26790.8 #Type your value as a floating point number

In [ ]:
# We are running alexnet benchmarks in isolation from the data loading and augmentation process
!mpirun -np 4 --allow-run-as-root -H localhost:4 -bind-to none -map-by slot -x NCCL_DEBUG=INFO -x PATH -mca pml ob1 -mca btl ^openib python3 cnn/alexnet.py --precision=fp16 --batch_size=512 --data_dir=/dli/data/tfdata352p90per2

In [ ]:
alexnet_pipeline = 3250.0 #Type your value as a floating point number

<h1>End of the optional segment</h1>

<h2> Plotting the GPU performance vs Pipeline Performance</h2>

<div>The experiments that we have executed illustrate very clearly that the overall training performance is significantly affected by the performance of the end to end training system. The below plots replicate the findings presented during the lecture and demonstrate the extent factors external to the GPU affect performance. </div>

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
alexnetplt = plt.plot(alexnet_gpu,alexnet_pipeline,'co')
googlenetplt = plt.plot(googlenet_gpu,googlenet_pipeline,'ro')
resnet50plt = plt.plot(resnet50_gpu,resnet50_pipeline,'bo')
resnet152plt = plt.plot(resnet152_gpu,resnet152_pipeline,'yo')
plt.axis('equal')
plt.show()

In [ ]:
plt.bar([1,2,3,4],[alexnet_pipeline/alexnet_gpu,
                   googlenet_pipeline/googlenet_gpu,
                   resnet50_pipeline/resnet50_gpu,
                   resnet152_pipeline/resnet152_gpu])
plt.show()

<div>It is important to note that the code that we have used implements a state of the art pipeline. We did not deliberately choose implementation which is inefficient. On the contrary, the presented code underwent man years of optimisation to deliver the results presented. Any inefficiencies introduced by customisations to this code will have a further impact on the performance.</div>

<h2> Conclusions </h2>

<div>As you could clearly see the performance of your model is significantly affected by the performance of your I/O pipeline. The problem is proportional to the compute required to process a portion of the data. The more compute is required the more time we have to prepare the next portion. This has several very important implications:
<ul>
        <li>As we now know the performance of CPUs is not growing proportionally to the Moore's Law. Therefore, as the GPUs increase in computational capability, it is not being matched by the CPU. <img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task3_img_MooresLaw.png" width=300 height=300/> This is a problem for our data loading and augmentation pipeline as the CPU gets less and less time to process the fixed amount of data. The below image illustrates a more comprehensive rooftop analysis discussed in detail during the lecture. As you can see you are seeing consistent results. <img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task3_img_RooflineAnalysis.png" width=400 height=400/>
    </li>
    <li>In quite a few application areas deep learning models need to fulfill strict non-functional requirements related to latency and overall compute. For example, in self-driving car problem space, we have a very constrained computational and power budget and need to operate with a certain latency (think of breaking). Those problems will use models which use less compute per portion of data as a consequence making them sensitive to any inefficiencies in the I/O pipeline. </li>
    <li>It becomes straightforward to design artificial algorithms for benchmarking I/O pipeline performance. The example presented below is an implementation of a neural network which has very low computational cost therefore, exaggerating any I/O inefficiencies. </li>
    </ul>

</div>

In [ ]:
!mpirun -np 4 --allow-run-as-root -H localhost:4 -bind-to none -map-by slot -x NCCL_DEBUG=INFO -x PATH -mca pml ob1 -mca btl ^openib python3 cnn/trivial.py --precision=fp16 --batch_size=512

In [ ]:
gpuOnlyReferencePerformance=53000.0 

In [ ]:
!mpirun -np 4 --allow-run-as-root -H localhost:4 -bind-to none -map-by slot -x NCCL_DEBUG=INFO -x PATH -mca pml ob1 -mca btl ^openib python3 cnn/trivial.py --precision=fp16 --batch_size=512 --data_dir=/dli/data/tfdata352p90per2

In [ ]:
referencePerformance=3200.0

In [ ]:
degradation = referencePerformance / gpuOnlyReferencePerformance
print("Our performance dropped to N%:")
print(degradation*100)

<div>We will use this neural network architecture to illustrate key engineering challenged related to data loading and augmentation.</div>

<h1> Data loading and augmentation </h1>

<div>The below graph illustrates a simple data loading and augmentation pipeline: </div>

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task3_img_Queue.png" width=600 height=600/> 

<div>Let's go through all of the steps of the process and analyse the way they affect the end to end system performance. Let's start with the storage mechanism.</div>

<h2> Storage </h2>

<div> Do you think that the observed drop in performance was related to the speed of the data storage? </div>

<div>Lets test that. Please open a new terminal window in Jupyter notebook by clicking here: <a href="../../../../terminals/1">terminal</a>
<br/><br/>
    
You should see the following window: <br/><img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task3_img_JupyterTerminal.png" width=400 height=400/>
    
</div>

<div>Enter the following command: "/dli/tasks/task3/task/nmon16e_x86_ubuntu1510" what will execute a resource monitor which we will use to monitor the disk, memory and CPU utilization.

You should be presented with the following screen: <br/><img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task3_img_JupyterNMON.png" width=400 height=400 />
</div>

<div>In order to start monitoring the disk utilisation press "d". You will be presented with the following screen: <br/><img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task3_img_nmonDisk.png" width=400 height=400/>
<br/>
    
Now align both screens side by side so that you can observe the progress on both and execute the below command again. What happens to the disk utilisation as you progress through the training process?
</div>

In [ ]:
!mpirun -np 4 --allow-run-as-root -H localhost:4 -bind-to none -map-by slot -x NCCL_DEBUG=INFO -x PATH -mca pml ob1 -mca btl ^openib python3 cnn/trivial.py --precision=fp16 --batch_size=512 --data_dir=/dli/data/tfdata352p90per2

<div> As you will see in the resource monitor the disks were not used at all. How is that possible? In our case, the dataset was fairly small (a tiny subset of the ImageNet dataset) and the system you are working on is equipped with a fair amount of RAM which was used by the Linux kernel to cache the frequently used data. In fact, our dataset is so small that it barely makes a dent in the memory resource available: <img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task3_img_nmonMemory.png" width=400 height=400/> </div>

<div>This does not mean that the speed of your storage mechanism is irrelevant for the overall performance. It just means that for the datasets which are smaller than the cache capability of the local machine you will pay the data access price just once during the first epoch and from that point on you operate at the speed of local cache (so either RAM or a speed of a dedicated SSD RAID array). </div>
    
<div>For the datasets which are bigger than the local cache (4/8/16 TB) the performance of the storage mechanism as well as the networking technology responsible for data delivery becomes critical for success. </div> 

<div>The secondary finding of this experiment is the fact that we are bound by other factors related to data loading and augmentation. Let us repeat the exercise above again this time observing the CPU utilisation rather than the disk utilisation. To turn off the disk monitor press "d" again. To start a CPU monitor press "c". You should be presented with the following screen: <img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task3_img_nmonCPU0.png" width=400 height=400/></div>

In [ ]:
!mpirun -np 4 \
  --allow-run-as-root \
  -H localhost:4 \
  -bind-to none -map-by slot \
  -x NCCL_DEBUG=INFO -x PATH \
  -mca pml ob1 -mca btl ^openib \
  python3 cnn/trivial.py --precision=fp16 --batch_size=512 --data_dir=/dli/data/tfdata352p90per2

In [ ]:
referencePerformance = 3550.5 # Please enter the performance metric you have obtained using a floating point number

<div>What you should have observed is a significant load on the CPU <br/><img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task3_img_nmonCPU.png" width=400 height=400/> <br/>In the next section, we will investigate a wider set of factors that affect the efficiency of your training process and your ability to scale the training process linearly.</div>

<h2> Loading </h2>

<div>Let's talk about the impact of inefficiencies of the I/O pipeline. As discussed in the lecture, when implementing a data queuing mechanism the neural network training process is no longer sensitive to data read latencies and its efficiency becomes directly a function of data throughput. Building a loading pipeline that can effectively load and process the data is a nontrivial engineering challenge. Fortunately, the majority of the deep learning frameworks already come with an efficient data loading mechanism. This also includes TensorFlow which comes equipped with a wide range of efficient data reading, transformation, and queuing mechanisms. The figure below taken from the TensorFlow best practices article illustrates the principles involved in <a href="https://www.tensorflow.org/api_guides/python/reading_data">loading the data</a> visually.

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task3_img_AnimatedFileQueues.gif" width=600/></div>

<div>The code implementing our benchmarking neural network also takes advantage of the TensorFlow data loading functionality. In particular, the reading step takes advantage of the "data_flow_ops.RecordInput" function which spawns multiple readers interacting with the data across multiple threads. Following that the data is decoded using the "INTEGER_FAST" implementation in the "tf.image.decode_jpeg" function. This exercise will introduce inefficiencies into the pipeline, i.e. will reduce the number of reading threads and the algorithm used for JPEG decoding to illustrate the effect it will have for the overall performance.</div>
<br/>
<div>Let us start by executing the code with the decreased number of reader threads.</div>

In [ ]:
!mpirun -np 4 \
  --allow-run-as-root \
  -H localhost:4 \
  -bind-to none -map-by slot \
  -x NCCL_DEBUG=INFO -x PATH \
  -mca pml ob1 -mca btl ^openib \
  python3 cnn1Thread/trivial.py --precision=fp16 --batch_size=512 --data_dir=/dli/data/tfdata352p90per2

In [ ]:
oneThrPerformance = 3178.2 # Please enter the performance metric you have obtained using a floating point number

<div>Now lets, execute the code with the suboptimal JPEG decoding implementation.</div>

In [ ]:
!mpirun -np 4 \
  --allow-run-as-root \
  -H localhost:4 \
  -bind-to none -map-by slot \
  -x NCCL_DEBUG=INFO -x PATH \
  -mca pml ob1 -mca btl ^openib \
  python3 cnnjpegLowPerformance/trivial.py --precision=fp16 --batch_size=512 --data_dir=/dli/data/tfdata352p90per2

In [ ]:
jpegLowPerformance = 3394.3 # Please enter the performance metric you have obtained using a floating point number

Finally, let us investigate the cumulative effect of both changes.

In [ ]:
!mpirun -np 4 \
  --allow-run-as-root \
  -H localhost:4 \
  -bind-to none -map-by slot \
  -x NCCL_DEBUG=INFO -x PATH \
  -mca pml ob1 -mca btl ^openib \
  python3 cnnjpegLowPerformance1Thread/trivial.py --precision=fp16 --batch_size=512 --data_dir=/dli/data/tfdata352p90per2

In [ ]:
jpegCumulativePerformance = 2940.4 # Please enter the performance metric you have obtained using a floating point number

In [ ]:
print(referencePerformance)
print(oneThrPerformance)
print(jpegLowPerformance)
print(jpegCumulativePerformance)

<div>As we can see, even though our dataset reside entirely in computer memory, the cumulative effect of the changes introduced on the overall training performance is nontrivial. The effect is even more apparent where more fundamental changes are introduced. For example its very easy to use even less optimal implementation of the data loading logic (to the extent where the system cannot take advantage of the cached files) making a much more substantial impact on the performance. The figure listed below illustrates this impact in more detail.
<br/>
<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task3_img_Loading.png" width=300 height=300/><br/>
</div>
<div>In many applications areas, this impact is so substantial that it makes sense to migrate elements of the data loading and augmentation pipeline to the GPU. This is not always a trivial task as some aspects of the process (such as JPEG decoding) are inherently serial taking limited advantage of the multithreaded nature of the GPU. As GPU performance increases this is to a large extent an unavoidable task. Training even a modestly sized network such as ResNet 50 on 8 P100 GPUs with as simple data loading and augmentation pipeline as described at the beginning of this section can easily saturate an entire state of the art CPU socket.</div><br/>

<h2> Augmentation </h2>

<div> Data augmentation is an important technique which, when done correctly, allows to significantly extend the available training dataset and address challenges related to rare or complex to capture classes of problems. As the name suggests the process involves augmenting the existing dataset with representative noise. For example when building speech recognition system one could overlay conversation or car noise to the existing recordings. Similarly, when working with images one could rotate them or change the colours ever so slightly. Those techniques expose our neural network to the richer dataset and lead to overall better neural network performance. As you can imagine they do come at a nontrivial computational cost.</div>
<br/>
<div>Since augmentation is a very common technique, the majority of the deep learning frameworks will come with a very rich ecosystem of utilities to support it. TensorFlow is no different. Our example code uses a fairly standard augmentation pipeline which involves:
    <ul>
        <li>Random cropping and resizing the image through "tf.image.sample_distorted_bounding_box"</li>
        <li>Image colour distortion through changes of the brightness ("tf.image.random_brightness"), saturation ("tf.image.random_saturation"), hue ("tf.image.random_hue") and contrast ("tf.image.random_contrast")</li>
        <li>Random flipping the image ("tf.image.random_flip_left_right")</li>
    </ul>
    
In this exercise we will create two variants of the code:
<ul>
    <li>First one that extends the pipeline by adding the "tf.image.random_flip_up_down" step</li>
    <li>Second one that simplifies the pipeline by removing the "tf.image.random_flip_left_right" step</li>
</ul></div>

In [ ]:
!mpirun -np 4 \
  --allow-run-as-root \
  -H localhost:4 \
  -bind-to none -map-by slot \
  -x NCCL_DEBUG=INFO -x PATH \
  -mca pml ob1 -mca btl ^openib \
  python3 cnnComplexAugmentation/trivial.py --precision=fp16 --batch_size=512 --data_dir=/dli/data/tfdata352p90per2

In [ ]:
complexAugmentationPerformance = 2579.9

In [ ]:
!mpirun -np 4 \
  --allow-run-as-root \
  -H localhost:4 \
  -bind-to none -map-by slot \
  -x NCCL_DEBUG=INFO -x PATH \
  -mca pml ob1 -mca btl ^openib \
  python3 cnnSimpleAugmentation/trivial.py --precision=fp16 --batch_size=512 --data_dir=/dli/data/tfdata352p90per2

In [ ]:
simpleAugmentationPerformance=3670.1

In [ ]:
print(referencePerformance)
print(complexAugmentationPerformance)
print(simpleAugmentationPerformance)

<div>As you can see, adding or removing additional augmentation steps substantially affected the end to end performance. This behavior (so when you are bottlenecked by the CPU resource) is common for a wide range of problems. That is why NVIDIA is working on a project DALI which intends not only to migrate this workload to the GPU but also standardize this part of the process across Deep Learning Frameworks.</div>

<div> NVIDIA Data Loading Library (<a href="https://docs.nvidia.com/deeplearning/sdk/dali-developer-guide/docs/index.html">NVIDIA DALI</a>) is a collection of highly optimized building blocks and an execution engine to accelerate input data pre-processing for deep learning applications. DALI provides both performance and flexibility of accelerating different data pipelines, as a single library, that can be easily integrated into different deep learning training and inference applications.<br/>

Key highlights of DALI include:
<ul>
<li>Full data pipeline accelerated from reading from disk to getting ready for training/inference</li>
<li>Flexibility through configurable graphs and custom operators</li>
<li>Support for image classification and segmentation workloads</li>
<li>Ease of integration through direct framework plugins and open source bindings</li>
<li>Portable training workflows with multiple input formats - JPEG, LMDB, RecordIO, TFRecord</li>
<li>Extensible for user specific needs through open source license</li>
    </ul></div>

<h1> Connectivity </h1>

<div>The connectivity between the host system and the GPU is critical for efficient data delivery. Similarly, for the jobs that span across multiple GPUs in the system, the GPU to GPU communication is critical as it enables the exchange of gradient information between the devices. Finally, if your job spans across multiple machines in the cluster or your dataset does not reside together with the compute (the majority of interesting use cases) the external networking becomes a critical component affecting the performance. When you bear that in mind the design principles of modern AI focused hardware become obvious: <br/><img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task3_img_NVLINK.png" width=500/>

<ul>
    <li>The PCIe bus on many occasions becomes a major bottleneck when it comes to effective scaling. Especially on the systems with multiple GPU it can get significantly saturated just by data delivery leaving no room for the gradient exchange. Moreover, on systems with more than one PCIe complex, the interconnection between the CPUs (QPI) becomes a major bottleneck. In order to address that NVIDIA has developed a dedicated bus that allows us to exchange the information between the GPUs without putting any pressure on the PCIe or the host system. In the diagram listed above, you see one of the most common at this point of time network topologies used with NVLINK - the cube mesh. <br/><img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task3_img_NVLINK2.png" width=300/></li>
    <li>This does not mean that the architecture of the PCIe becomes irrelevant. On the contrary, having a balanced PCIe architecture is critical for efficient intra and extra node scaling. <br/><img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task3_img_AllReduceBandwidth.png" width=600/></li>
    <li>Finally having four PCIe switches together with four InfiniBand adapters allows the GPUs to communicate between nodes using Direct Memory Access technology not placing any overheads on the CPU or memory of the host devices and significantly reducing latency and increasing throughput.</li>
</ul>
    
</div>

<br/>

<div>Let's investigate the properties of the PCIe and NVLINk busses.</div>

<h2> PCIe </h2>
<div> Let's start by investigating the communication topology of our DLI machine by executing the following snippet of code:</div>

In [ ]:
!nvidia-smi topo -m

As you can see our system is equipped with four GPUs which are interconnected using both the PCIe and the NVLINK buss. Let us start by executing some code benchmarking the current PCIe system performance.

In [ ]:
# Building the benchmarking code
!nvcc bandwidthtest.cu -o bandwidthtest

In [ ]:
# Executing the PCIe benchmark
!nvprof ./bandwidthtest

<div>The exact results will vary but on my machine, the Host to Device performance peaked at 11.8 GB/s. How does it compare to the data volumes we need to transport? Let's start by measuring the size of our experimental dataset.</div>

In [ ]:
# Let's start by measuring the size of our experimental dataset
!du -sh /dli/data/tfdata352p90per2

For this exercise, the ImageNet dataset was substantially stripped down leaving only one image per category. Therefore, the dataset contains 236 megabytes of images. Let’s calculate how much data do we need to deliver to a GPU every second (bear in mind we are using some of the data collected in the previous part of this Lab).

In [ ]:
# Lets calculate how much data do we need to deliver to a GPU every second
imageWidth = imageHeight = 352
numberOfChannels = 3.0
imageSizeMB = imageWidth*imageHeight*numberOfChannels/1024/1024
print("Single image size [MB]:")
print(imageSizeMB)
totalBandwidthRequiredForDataDeliveryOnlyGB = gpuOnlyReferencePerformance*imageSizeMB/1024
print("Bandwidth required to deliver data to a single GPU for the current neural network [GB/s]:")
print(totalBandwidthRequiredForDataDeliveryOnlyGB)
totalBandwidthRequired8GPUGB = totalBandwidthRequiredForDataDeliveryOnlyGB*8

print("Bandwidth required to deliver data to a eight GPUs for the current neural network [GB/s]:")
print(totalBandwidthRequired8GPUGB)

<div>As you can see, with this example, the PCIe bus does not have enough bandwidth even for data delivery to our neural network (do bear in mind that we have chosen an architecture aiming to stress the I/O pipeline). Not to mention any spare bandwidth for the gradient information exchange. </div>
<br/>
<div>How much data do we need to exchange to allow for the gradient updates? The detailed calculation is beyond the scope of this lecture (refer to the <a href="https://www.youtube.com/watch?v=vT1JzLTH4G4&list=PL3FW7Lu3i5JvHM8ljYj-zLfQRF3EO8sYv">Stanford's CS231n course on computer vision</a>) but for a ResNet 50 network, when using an all reduce algorithm, we will need to exchange approximately 97 MB of data (as it has 25610216 parameters 32 bit each) twice (as we need to both receive the gradient and send it out) within the time it takes to do a backpropagation. As we saw earlier we were calculating 4 batches of data every second. This means that we have approximately 125 ms to exchange the gradient information leading to peer to peer bandwidth requirement of 1552 MB/s (and substantially more if a shared medium needs to be used). </div>

<h2> NVLINK </h2>

This was one of the key reasons for the development of the NVLINk bus interconnecting the GPUs in a system. In the next section, we will look at its performance when combined with the NCCL based all reduce algorithm and once again look at the PCIe comparison. We will close this lab by looking at a code snippet illustrating the use of NCCL (and as a consequence NVLINK).

<h3>NVLINK performance</h3>

Let’s look at the NVLINK bus performance by executing a test script downloaded directly from the NVIDIA repository: <a href="https://github.com/nvidia/nccl-tests">NCCL-tests</a>. The code snippet listed below will execute several all-reduce operations exchanging information between four Volta V100 GPUs available in this DLI machine. As you will see we run the script for messages of varying size from 8 bytes to 512 megabytes. The test will generate several performance metrics but the one we are interested in is the achieved buss bandwidth (busbw) and algorithm bandwidth (algbw) in GB/s. 

If you recall the way our data parallel implementation of SGD worked, once we have calculated the gradient values for every GPU participating in the process we then wanted to calculate its average value. As you will recall this can be done also with the use all the all reduce algorithm hence the below example. Please execute the below script now to observe the performance of the real performance of the communication (effective bandwidth that was achieved for the all reduce operation across all 4 devices).


In [ ]:
!./all_reduce_perf -b 8 -e 512M -f 2 -g 4

Let's now disable the Peer to Peer communication (both on NVLINK and PCIe) and fall back to IPC for data exchange (so having to do a round trip through the host CPU and memory).  We achieve that by setting the NCCL_P2P_DISABLE environmental variable to 1.

In [ ]:
!NCCL_P2P_DISABLE=1 ./all_reduce_perf -b 8 -e 512M -f 2 -g 4

How did this affect the performance of the all-reduce algorithm? What do you think will be the impact of extending the system to 8 GPUs (from the current 4 GPU setup) and adding a CPU connected using the QPI link?

<h3>Horovod and NVLINK implementation</h3>
Now that we have discussed the merits of using the NVLINK bus and the NCCL library for the all-reduce implementation let us have a look on the output NCCL library has created during the execution of our training jobs.<br/><br/>

The image below illustrates the communication topology created by the NCCL library on a DGX-2 system. The output lists GPUs from 0 to 15 as this is a 16 GPU system. We can see two distinct communication rings created. In this case, the topology spans exclusively across the GPUs within our training system but as discussed in the lecture, NCCL can also create distributed rings across multiple distributed systems (using RDMA GPU Direct technology).
<br/><img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task3_img_RingsAmong2DGX.png" width=600/><br/>

Please refer to the output of the training cycle and inspect the NCCL ring topology generated for your training job. 
<br/><br/>
Secondly, as discussed in the lecture, Horovod allows us to ignore the implementation detail of the NCCL library. The communication is managed behind the scenes by the distributed optimizer. Saying that this functionality can be addressed directly in TensorFlow and is currently implemented under <a href="https://www.tensorflow.org/api_docs/python/tf/contrib/nccl">tensorflow.contrib.nccl</a>

# Final Exercise: Accelerate on 4 GPUs

The goal of this exercise is to test your capability to work with the fundamental features of Horovod. You will be presented with a single GPU implementation of a Neural Network and will be asked to introduce minimal changes to that code so that it can be distributed using Horovod. In order to do that please modify the [code](../../../../edit/tasks/task3/task/assessmentBaseHvd.py) by filling in all the **##TODO##** sections in the file.

To test the single GPU implementation (prior to introducing changes), execute the following command:

In [ ]:
!python3 assessmentBaseHvd.py

Test your modified code on 4 GPUs by using the following mpirun command:

In [ ]:
!mpirun --allow-run-as-root -np 4 python3 assessmentBaseHvd.py